# 5) Assign BGs to Zones

## Env

In [2]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
from keplergl import KeplerGl
import pickle
import os


### Get Pickles

In [3]:
# Read GEO_DEFINITIONS
with open('../data/temp/geo_definitions', 'rb') as fp:
    GEO_DEFINITIONS = pickle.load(fp)

# gdf BG centroids
with open('../data/temp/gdf_bg_centroids', 'rb') as f:
    gdf_BG_centroids = pickle.load(f)

# Load the projections
with open('../data/temp/projections', 'rb') as f:
    PROJ = pickle.load(f)

# Zones
zones = {}
shortest_paths = {}
#for m in GEO_DEFINITIONS.keys():
for m in ['chi','phi','sfba','nyc']:
    print(f'{m}..')
    with open(f'../data/temp/zones_{m}', 'rb') as fp:
        zones[m] = pickle.load(fp)

    # Shortest Paths
    with open(f'../data/temp/bg_dists_{m}', 'rb') as fp:
        shortest_paths[m] = pickle.load(fp)

chi..
phi..
sfba..
nyc..


## Join BG -> Zone

In [4]:
bg2zones = {}

for m in ['chi','phi','sfba','nyc']:
    print(f'{m}..')
    bg2zones[m] = (
        gdf_BG_centroids[gdf_BG_centroids['metro'] == m]
        .to_crs(PROJ[m])
        .sjoin(
            zones[m]
            .reset_index()
            .drop('index',axis=1)
        )
        .rename(columns={'index_right':'zoneID'})
    )

chi..
phi..
sfba..
nyc..


## Join BG2Zone -> BG OD Pairs

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [46]:

final_dataset = {}
for metro in ['chi','sfba','nyc','phi']:
    print(f'{metro}..')
    final_dataset[metro] = (
        shortest_paths[metro]
        .reset_index()
        .rename(columns={'value':'dist'})
        .merge(
            bg2zones[metro][['fips12','zoneID']],
            left_on = 'fips12_o',
            right_on = 'fips12',
            suffixes = ('','_o')
        )
        .drop('fips12', axis=1)
        .merge(
            bg2zones[metro][['fips12','zoneID']],
            left_on = 'fips12_d',
            right_on = 'fips12',
            suffixes = ('_o','_d')
        )
        .drop('fips12', axis=1)
        .assign(zone_same = lambda x: x.zoneID_o == x.zoneID_d)
    )

#(
    
    #bg2zones['chi']

chi..
sfba..
nyc..
phi..


In [62]:
for m in ['sfba','chi','phi','nyc']:
    print(m)
    display(final_dataset[m].groupby('zone_same').agg({'fips12_o':'count', 'dist':'mean'}))

sfba


,fips12_o,dist
zone_same,,
False,45875,3038.196557
True,192055,2439.553043


chi


,fips12_o,dist
zone_same,,
False,35416,3102.469449
True,201237,2511.951823


phi


,fips12_o,dist
zone_same,,
False,24250,3036.950452
True,137609,2464.998227


nyc


,fips12_o,dist
zone_same,,
False,351700,2932.748430
True,1283053,2380.060433


## Connect to postgres

In [63]:
# Postgres
from dotenv import load_dotenv

load_dotenv()

from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text
# os.chdir('Documents/Grad School/Berkeley/Classes/CYP_204D - Multivariate Analysis in Planning/Project/204d_final_project/')
user = "postgres"
password = os.getenv('POSTGRES_PW')
host = "127.0.0.1"
port = "5432"
database = "gradschool"

pg_url = URL.create(
    drivername='postgresql',
    username='postgres',
    host='127.0.0.1',
    database='gradschool',
    password=os.getenv('POSTGRES_PW')
)

pg_engine = create_engine(pg_url)
# Test connection
try:
    with pg_engine.connect() as connection_str:
        print('Successfully connected to the PostgreSQL database')
except Exception as e:
    print(f'Sorry failed to connect: {e}')

pg_connection = pg_engine.connect()


Successfully connected to the PostgreSQL database


In [65]:
for m in ['sfba','chi','phi','nyc']:
    final_dataset[m].to_sql(f'{m}_bg2zones_od_dists', con=pg_connection, schema='cyp204d_final_project')

In [1]:
zones

NameError: name 'zones' is not defined

# Redo with raw data

In [9]:
os.getcwd
# df_counts_raw = {}
# DATA_PATHS = {
#     'phi' : '../data/replica_exports/replica-phi_sat_spring_2024-01_14_25-trips_dataset.csv',
#     'sfba' : '../data/replica_exports/replica-ca_sat_spring_2024-01_14_25-trips_dataset.csv',
#     'chi' : '../data/replica_exports/replica-chi_sat_spring_2024-01_14_25-trips_dataset.csv',
#     'nyc' : '../data/replica_exports/replica-ny_sat_spring_2024-01_14_25-trips_dataset.csv'
# }
# df_counts = {}
# for m in DATA_PATHS.keys():
#     df_counts_raw[m] = pd.read_csv(DATA_PATHS[m])
#     df_counts[m] = df_counts_raw[m][['origin_bgrp_fips_2020', 'destination_bgrp_fips_2020', 'trip_distance_meters']]


['.DS_Store',
 'cache',
 'code',
 'ReadMe.md',
 'viz',
 '.gitignore',
 '.env',
 '.git',
 'data']

In [ ]:
for m in df_counts['nyc']:
    m.join()